In [1]:
import requests, json
from bs4 import BeautifulSoup
from urllib.parse import urlencode


In [2]:
amphibiaweb_url = 'https://amphibiaweb.org/'

In [3]:
def getSpeciesData(spname, siteurl):
    search_baseurl = 'cgi/amphib_query?'
    d = { 'where-scientific_name':spname }
    url = siteurl+search_baseurl+urlencode(d)
    return requests.get(url)

In [4]:
def getSpeciesIUCN(httpresp):
    bs = BeautifulSoup(httpresp.text)
    return bs.find('table', {'class':'listing'}).findNext('td').findNext('td').get('class')[0]

In [6]:
with open('../_data/species.json', 'r') as f:
    splist = [ i['scientificName'] for i in json.load(f) ]

In [55]:
def parseSpeciesNames(splist, siteurl):
    res = {'ok':[], 'failed':[]}
    for sp in splist:        
        try: 
            bs = BeautifulSoup(getSpeciesData(sp, siteurl).text)
            nameFound = bs.find('div',{'id':'main'}).find('font').text
            if nameFound == sp:
                res['ok'].append(sp)
        except Exception as e: 
            res['failed'].append(sp)
        
    return res

In [58]:
j = parseSpeciesNames(splist, amphibiaweb_url)

In [60]:
j['failed']

['Boana albopunctatus',
 'Boana goianus',
 'Boana sp.',
 'Pithecopus hypochondrialis',
 'Pithecopus oreades',
 'Scinax aff. squalirostris',
 'Scinax rupestris',
 'Leptodatcylus syphax',
 'Leptodactylus cf. cunicularius',
 'Pysalaemus cuvieri',
 'Elachistocleis sp.']

In [53]:
j = []
for sp in splist:
    j += [parseSpeciesNames(sp, amphibiaweb_url)]

In [54]:
j

[{'ok': ['Allobates goianus'], 'failed': []},
 {'ok': ['Rhaebo guttatus'], 'failed': []},
 {'ok': ['Rhinella mirandaribeiroi'], 'failed': []},
 {'ok': ['Rhinella rubescens'], 'failed': []},
 {'ok': ['Rhinella schneideri'], 'failed': []},
 {'ok': ['Barycholos ternetzi'], 'failed': []},
 {'ok': ['Ameerega flavopicta'], 'failed': []},
 {'ok': ['Aplastodiscus lutzorum'], 'failed': []},
 {'ok': ['Bokermannohyla pseudopseudis'], 'failed': []},
 {'ok': ['Dendropsophus jimi'], 'failed': []},
 {'ok': ['Dendropsophus minutus'], 'failed': []},
 {'ok': ['Dendropsophus rubicundulus'], 'failed': []},
 {'ok': [], 'failed': ['Boana albopunctatus']},
 {'ok': ['Boana ericae'], 'failed': []},
 {'ok': [], 'failed': ['Boana goianus']},
 {'ok': ['Boana lundii'], 'failed': []},
 {'ok': ['Boana phaeopleura'], 'failed': []},
 {'ok': ['Boana raniceps'], 'failed': []},
 {'ok': [], 'failed': ['Boana sp.']},
 {'ok': [], 'failed': ['Pithecopus hypochondrialis']},
 {'ok': [], 'failed': ['Pithecopus oreades']},
 {'ok

In [7]:
l = []
for sp in splist:
    try:
        resp = getSpeciesData(sp, amphibiaweb_url)
        iucn_cat = getSpeciesIUCN(resp)
        l += [ (sp,iucn_cat) ]
        print(f"{sp} at category {iucn_cat}")
    except:
        print(f"Could not find name: {sp}")
        l += [ (sp, None) ]

Allobates goianus at category DD
Rhaebo guttatus at category LC
Rhinella mirandaribeiroi at category 
Rhinella rubescens at category LC
Rhinella schneideri at category LC
Barycholos ternetzi at category LC
Ameerega flavopicta at category LC
Aplastodiscus lutzorum at category 
Bokermannohyla pseudopseudis at category LC
Dendropsophus jimi at category LC
Dendropsophus minutus at category LC
Dendropsophus rubicundulus at category LC
Could not find name: Boana albopunctatus
Boana ericae at category DD
Could not find name: Boana goianus
Boana lundii at category LC
Boana phaeopleura at category DD
Boana raniceps at category LC
Could not find name: Boana sp.
Could not find name: Pithecopus hypochondrialis
Could not find name: Pithecopus oreades
Could not find name: Scinax aff. squalirostris
Scinax fuscomarginatus at category LC
Scinax fuscovarius at category LC
Scinax rogerioi at category 
Scinax similis at category LC
Scinax skaios at category 
Scinax tigrinus at category LC
Could not find n

In [10]:
l

[('Allobates goianus', 'DD'),
 ('Rhaebo guttatus', 'LC'),
 ('Rhinella mirandaribeiroi', ''),
 ('Rhinella rubescens', 'LC'),
 ('Rhinella schneideri', 'LC'),
 ('Barycholos ternetzi', 'LC'),
 ('Ameerega flavopicta', 'LC'),
 ('Aplastodiscus lutzorum', ''),
 ('Bokermannohyla pseudopseudis', 'LC'),
 ('Dendropsophus jimi', 'LC'),
 ('Dendropsophus minutus', 'LC'),
 ('Dendropsophus rubicundulus', 'LC'),
 ('Boana albopunctatus', None),
 ('Boana ericae', 'DD'),
 ('Boana goianus', None),
 ('Boana lundii', 'LC'),
 ('Boana phaeopleura', 'DD'),
 ('Boana raniceps', 'LC'),
 ('Boana sp.', None),
 ('Pithecopus hypochondrialis', None),
 ('Pithecopus oreades', None),
 ('Scinax aff. squalirostris', None),
 ('Scinax fuscomarginatus', 'LC'),
 ('Scinax fuscovarius', 'LC'),
 ('Scinax rogerioi', ''),
 ('Scinax similis', 'LC'),
 ('Scinax skaios', ''),
 ('Scinax tigrinus', 'LC'),
 ('Scinax rupestris', None),
 ('Trachycephalus typhonius', 'LC'),
 ('Adenomera cotuba', ''),
 ('Adenomera juikitam', ''),
 ('Adenomera s

In [11]:
[ i[0] for i in l if i[1] is None and '.' not in i[0] ]

['Boana albopunctatus',
 'Boana goianus',
 'Pithecopus hypochondrialis',
 'Pithecopus oreades',
 'Scinax rupestris',
 'Leptodatcylus syphax',
 'Pysalaemus cuvieri']

## Get text from wix

In [95]:
url = 'https://www.lafuc.com/rhinella-rubescens'

bs = BeautifulSoup(requests.get(url).text)

In [25]:
container = bs.find('div', {'id': 'PAGES_CONTAINER'})

([ i.text for i in container.findAll('p') ]

SyntaxError: unexpected EOF while parsing (<ipython-input-25-7fa49a152a07>, line 2)

## Glossary

#### Create the glossary 

In [222]:
import re
from unidecode import unidecode

In [224]:
preprocess_terms = lambda x: re.sub('\s+','-', unidecode(x.strip())).lower()

In [223]:
with open('glossario.txt') as f:
    text = ''.join( l for l in f.readlines() if l!='\n')
    terms_defins = re.findall('([\w\s]+):(.+[^\n])', text)

In [229]:
glossary = { preprocess_terms(term): defin.strip() for term,defin in terms_defins }

In [230]:
glossary

{'poleiro': 'Local onde o animal fica e canta. Pode ser no chão, em um galho, na árvore, entre outros locais.',
 'matas-de-galeria': 'É um tipo de fitofisionomia do Bioma Cerrado, é um ambiente de formação florestal, cercado por um ambiente savânico.',
 'defesa-passiva': 'Tipo de defesa que só é ativada quando estimulada mecanicamente/manualmente.',
 'regiao-loreal': 'É o “rostinho” do anuro, é a região delimitada pelos olhos, boca e focinho, sempre lateral.',
 'glandula-parotida': 'Glândula de veneno que fica no dorso, atrás da cabeça.',
 'glandula-tibial': 'Glândula de veneno presente na perna, na tíbia.',
 'endemico': 'Esse termo é usado quando uma espécie só ocorre naquele local.',
 'dissecacao': 'Evitar que os ovos percam água e desidratam.',
 'ovoide': 'Corpo em formato de ovo.',
 'aposematico': 'Cor de advertência, chamativa, que avisa algum perigo.',
 'epiteto-especifico': 'É a segunda palavra dada quando se escreve o nome da espécie, geralmente é uma palavra referente à uma ca

#### Open the file to make the substitutions

In [387]:
mytext= 'ada lana <glossario ovoide >Ovóidile</glossario> tentar'
t = mytext

In [338]:
m = re.search('<glossario(?P<term0>[\w\s\-_]+)?>(?P<term>.+)</glossario>',t)
m.group('term0')

' ovoide'

In [298]:
with open('../especies/odontophrynus-salvatori.md') as f:
    text = ''.join(f.readlines())

In [404]:
with open('../especies/ameerega-flavopicta.md') as f:
    text = ''.join(f.readlines())

In [405]:
def substituteDefinition( text, glossary ):
    matches = [ match for match in re.finditer('<glossario(?P<term0>[\w\s\-_]+)?>(?P<term>[\w\s\-]+)</glossario>',text) ]
    
    for m in matches:
        matchstr = m.group()
        term = m.group('term')

        term_key = m.group('term0')
        term_key = preprocess_terms(term_key) if term_key is not None else ''
        if term_key=='':
            term_key = preprocess_terms(term)
        dfn = glossary.get(term_key,'')
        substitute =f'''<dfn title="{dfn}">{term}</dfn>'''
        text = re.sub(matchstr,substitute, text)   
    
    return text

In [406]:
with open('../especies/ameerega-flavopicta.md','w') as f:
    #f.writelines(text)
    f.writelines(substituteDefinition(text,glossary))

In [395]:
substituteDefinition(text,glossary)

'\ufeff*Ameerega flavopicta* é um anuro que chama muita atenção! Sua coloração chamativa, denominada de coloração <dfn title="">aposemática</dfn>, avisa aos predadores potenciais (especialmente aves e mamíferos) que existem toxinas na pele desse simpático sapinho. O sapo obtêm essas toxinas através da dieta, composta basicamente por formigas e ácaros. Na época reprodutiva, os machos vocalizam durante o dia, em meio à pedras e à vegetação que margeiam riachos. Seu canto lembra assobios curtos e repetitivos, parecendo um grilo. O seu <dfn title="É a segunda palavra dada quando se escreve o nome da espécie, geralmente é uma palavra referente à uma característica do animal.">epíteto específico</dfn> (*flavopicta*) significa pontos amarelos, uma característica da espécie em vida.\n\n\n# Diagnose:\nCorpo preto com um par de listra amarela que vai desde a ponta do nariz, passa acima dos olhos e vai até a coxa, terminando com uma cor alaranjada. Possui pontos amarelos no dorso e nas pernas e s